In [1]:
import cv2 as cv
import numpy as np
import glob
import os
import imutils
from imutils.perspective import four_point_transform

In [2]:
def print_img(i,img):
    cv.imshow(str(i),img)
    cv.waitKey(0)
    cv.destroyAllWindows()

In [3]:
def find_sudoku(i): #returns an image of just the sudoky grid from an image containing a sudoku
    image = cv.imread(images[i])
    gray = cv.cvtColor(image, cv.COLOR_BGR2GRAY) #grayscale
    blurred = cv.GaussianBlur(gray, (11,11), 3) #blur
    #threshold to get grid
    thresh = cv.adaptiveThreshold(blurred, 255,cv.ADAPTIVE_THRESH_GAUSSIAN_C, cv.THRESH_BINARY, 11, 2)
    thresh = cv.bitwise_not(thresh)

    # find contours in the thresholded image
    cnts = cv.findContours(thresh.copy(), cv.RETR_EXTERNAL,cv.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    #sort by biggest contour first
    cnts = sorted(cnts, key=cv.contourArea, reverse=True)

    #get 3 biggest contours
    sudokuCnt = [None,None,None]

    counter=0
    for c in cnts:
        # approximate the contour
        peri = cv.arcLength(c, True)
        approx = cv.approxPolyDP(c, 0.02 * peri, True)
        # check to see if it has 4 points
        if len(approx) == 4:
            sudokuCnt[counter] = approx
            counter+=1

    # apply a four point perspective transform to original image
    # automaticaly sorts points
    sudoku = [None,None,None]
    sudoku[0] = four_point_transform(image, sudokuCnt[0].reshape(4, 2))
    sudoku[1] = four_point_transform(image, sudokuCnt[1].reshape(4, 2))
    sudoku[2] = four_point_transform(image, sudokuCnt[2].reshape(4, 2))


    return sudoku


In [4]:
def get_matrix(x): # returns a numerical matrix from an image of a sudoku
    matrix = np.empty([9,9], dtype=int)
    # match each sudoku with a template digit
    # for each number from 1 to 9 it finds the boxes in the sudoku grid that match the template
    for k in range(1,10):
        img_rgb = x.copy()
        img_gray = cv.cvtColor(img_rgb, cv.COLOR_BGR2GRAY)
        template = cv.imread(str(k)+'.jpg',0)
        w, h = template.shape[::-1]
        
        res = cv.matchTemplate(img_gray,template,cv.TM_CCOEFF_NORMED)
        threshold = 0.85
        #it saves the locations where the template and the grid match
        loc = np.where( res >= threshold)
        #removes locations that are too similar to each other (they are the same box)
        new_loc = [[loc[1][0],loc[0][0]]]
        for i in range(1,len(loc[0])):
            if(abs(loc[0][i-1]-loc[0][i])>20):
                new_loc.append([loc[1][i],loc[0][i]])
                
        #visually check
    #     for pt in new_loc:
    #         cv.rectangle(img_rgb, (pt[0],pt[1]), (pt[0] + w, pt[1] + h), (0,0,255), 1)
    #         cv.circle(img_rgb,  (pt[0],pt[1]), 4, (0,255,255), -1)

    
        # turn into matrix by transforming pixel distances to ordinal numbers
        for i in range(9):
            new_loc[i][1]=i
        new_loc.sort()
        for i in range(9):
            new_loc[i][0]=i
        for i in new_loc:
            aux=i[0]
            i[0]=i[1]
            i[1]=aux
        new_loc.sort()
        
        # add to matrix
        for i in new_loc:
            matrix[i[0]][i[1]]=k
    return matrix



In [5]:
def make_cube(x,y,z): # from 7 fixed points warped onto the template cube
    # cv.circle(template,  (297,234), 3,  (128,128,255), -1) #A
    # cv.circle(template,  (5,158), 3,  (128,128,255), -1) #B
    # cv.circle(template,  (5,460), 3,  (128,128,255), -1) #C
    # cv.circle(template,  (297,538), 3,  (128,128,255), -1) #D
    # cv.circle(template,  (563,388), 3, (128,128,255), -1) #E
    # cv.circle(template,  (563,82), 3, (128,128,255), -1) #F
    # cv.circle(template,  (272,1), 3,  (128,128,255), -1) #G
    w,h,_=x.shape
    pts1=np.float32([[272,1],[563,82],[5,158],[297,234]])#gfba
    pts2=np.float32([[5,158],[297,234],[5,460],[297,538]])#bacd
    pts3=np.float32([[297,234],[563,82],[297,538],[563,388]])#afde
    pts4=np.float32([[0,0],[w,0],[0,h],[w,h]])

    M1 = cv.getPerspectiveTransform(pts4,pts1)
    M2 = cv.getPerspectiveTransform(pts4,pts2)
    M3 = cv.getPerspectiveTransform(pts4,pts3)

    dst1 = cv.warpPerspective(x,M1,(600,600))
    dst2 = cv.warpPerspective(y,M2,(600,600))
    dst3 = cv.warpPerspective(z,M3,(600,600))
    dst=dst1+dst2+dst3
    return dst

In [6]:
def order(mx,my,mz,x,y,z): #by sudoku rules we know that this is the only check we have to make
    if np.all(mx[8]==my[0]):                   #  mx
        return mx,my,mz,x,y,z                  #  my mz
    if np.all(my[8]==mx[0]):
        return my,mx,mz,y,x,z
    if np.all(mz[8]==mx[0]):
        return mz,mx,my,z,x,y
    if np.all(mx[8]==mz[0]):
        return mx,mz,my,x,z,y
    if np.all(my[8]==mz[0]):
        return my,mz,mx,y,z,x
    if np.all(mz[8]==my[0]):
        return mz,my,mx,z,y,x
    return -1,-1,-1,-1,-1,-1
        
def answer(k,f):
    template = cv.imread('train\\cube\\template.jpg')
    #find the 3 sudoku grids
    x,y,z = find_sudoku(k)
    #get the matrixes for the 3 sudoku grids
    mx=get_matrix(x)
    my=get_matrix(y)
    mz=get_matrix(z)
    #order the matrixes and the sudoku grids
    mx,my,mz,x,y,z=order(mx,my,mz,x,y,z)
    
    #create the cube
    cube = make_cube(x,y,z)
    cv.imwrite(output_folder+images[k][10:-4]+'_result.jpg',cube)
    
    #create the document
    for i in mx:
        for j in i:
            f.write(str(j))
        f.write('\n')
    f.write('\n')
    for i in range(len(my)):
        for j in my[i]:
            f.write(str(j))
        f.write(' ')
        for j in mz[i]:
            f.write(str(j))
        if i<8:
            f.write('\n')

In [7]:
base_folder = "test\\cube\\" #change to "test\\cube\\" for test images
output_folder = "evaluation\\submission_files\\Partu_Ana_Maria_407\\cube\\"
n_images = glob.glob(os.path.join(base_folder,'*.jpg'))
images=[]
for i in n_images:
    if('_' in i):
        pass
    elif('template' in i):
        pass
    else:
        images.append(i)

In [8]:
for i in range(10):
    with open(output_folder+images[i][10:-4]+"_predicted.txt","w") as f:
        answer(i,f)